In [1]:
using BenchmarkTools, Compat, DataFrames, ForwardDiff

In [2]:
df = readtable("data/model_australia.txt", separator = ' ', header = false)

names!(df, [Symbol("C$i") for i in 1:4])

head(df)

,C1,C2,C3,C4
1,1,0,0,0
2,0,1,0,0
3,0,0,1,0
4,35,0,0,0
5,69,34,35,0
6,70,71,70,30


In [3]:
function f(β::Vector, model::Float64 = 0.0, n::Int64 = 210)
    i = 1
    while i <= n
        probability = individual(β, i)
        model += log(probability(β))
        i += 1
    end
    return model/n
end

f (generic function with 3 methods)

In [4]:
function individual(β::Vector, i::Int64)
    data = convert(Array, df[i*7-6:i*7-1, :])
    choices = convert(Array, df[i*7:i*7, :])
    alternatives = find(choices .== 0)
    choice = find(choices .== 1)[1]
    
    function utility(β::Vector, i::Int64)
        return dot(vec(data[:, i]), β)
    end
    
    function probability(β::Vector, t::Float64 = 0.0)
        c = utility(β, choice)
        for alternative in alternatives
            t += exp(utility(β, alternative)-c)
        end
        return 1/(1+t)
    end
    
    return probability
end

individual (generic function with 1 method)

In [5]:
function g(β::Vector)
    return ForwardDiff.gradient(f, β)
end

g (generic function with 1 method)

In [6]:
function g_test(β::Vector, t::Float64 = 0.0, n::Int64 = 210)
    for i = 1:n
        probability = individual(β, i)
        t += (1/probability(β))*ForwardDiff.gradient(probability, β)
    end
    return t/n
end

g_test (generic function with 3 methods)

In [7]:
function g!(β::Vector, storage::Vector)
    s = g(β)
    storage[1:length(s)] = s[1:length(s)]
end

g! (generic function with 1 method)

In [8]:
function g_test!(β::Vector, storage::Vector)
    s = g_test(β)
    storage[1:length(s)] = s[1:length(s)]
end

g_test! (generic function with 1 method)

In [9]:
function H(β::Vector)
    return ForwardDiff.hessian(f, β)
end

H (generic function with 1 method)

In [10]:
function H!(β::Vector, storage::Matrix)
    s = H(β)
    n, m = size(s)
    storage[1:n, 1:m] = s[1:length(s)]
end

H! (generic function with 1 method)

In [11]:
function newton(f::Function, g::Function, h::Function, β0::Vector,
        δ::Float64 = 1e-6, nmax::Int64 = 1000)
    k = 1
    β = β0
    n = length(β)
    δ *=δ
    H = eye(n)
    dfβ = ones(n)
    g(β, dfβ)
    while dot(dfβ, dfβ) > δ && k <= nmax
        k += 1
        g(β, dfβ)
        h(β, H)
        β -= H\dfβ
    end
    return β, k
end

newton (generic function with 3 methods)

In [12]:
@btime newton(f, g!, H!, zeros(6))

  194.171 ms (455434 allocations: 55.83 MiB)


([5.20744, 3.86904, 3.16319, 0.013287, -0.0961248, -0.0155015], 7)

In [13]:
@btime newton(f, g!, H!, ones(6))

  75.625 ms (175130 allocations: 21.11 MiB)


([NaN, NaN, NaN, NaN, NaN, NaN], 3)

In [14]:
@btime newton(f, g_test!, H!, zeros(6))

  216.247 ms (631071 allocations: 64.90 MiB)


([5.20744, 3.86904, 3.16319, 0.013287, -0.0961248, -0.0155015], 7)

In [15]:
@btime newton(f, g_test!, H!, ones(6))

  81.422 ms (250403 allocations: 25.00 MiB)


([NaN, NaN, NaN, NaN, NaN, NaN], 3)